In [1]:
library(rhdf5)
library(biomaRt)

In [2]:
path = "../hdf5_data/raw_melanomaS2.h5"

In [3]:
h5f = H5Fopen(path)

In [4]:
gene_ids = h5f$gene_attrs$gene_ids

In [5]:
ensembl = useMart("ensembl")
mart = useDataset("hsapiens_gene_ensembl", mart=ensembl)

In [6]:
mart

Object of class 'Mart':
  Using the ENSEMBL_MART_ENSEMBL BioMart database
  Using the hsapiens_gene_ensembl dataset

In [7]:
sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS: /cluster/work/bewi/members/tuncel/anaconda3/envs/r-3.4.1-env/lib/R/lib/libRblas.so
LAPACK: /cluster/work/bewi/members/tuncel/anaconda3/envs/r-3.4.1-env/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] biomaRt_2.34.2   rhdf5_2.22.0     RevoUtils_10.0.8

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.16         compiler_3.4.3       prettyunits_1.0.2   
 [4] bitops_1.0-6         tools_

In [8]:
entrezGeneMapping_proteinCoding = getBM(
  attributes= c("ensembl_gene_id", "entrezgene", "hgnc_symbol", "description"),
  filters = c("ensembl_gene_id","biotype"),
  values= list(gene_ids,'protein_coding'),
  mart= mart,
  uniqueRows=T)

ERROR: Error in dyn.load(file, DLLpath = DLLpath, ...): unable to load shared object '/cluster/work/bewi/members/tuncel/anaconda3/envs/r-3.4.1-env/lib/R/library/curl/libs/curl.so':
  libgfortran.so.4: cannot open shared object file: No such file or directory


In [22]:
# filter NAs and multiple entries
entrezGeneMapping_proteinCoding_noNa = na.omit(entrezGeneMapping_proteinCoding)
entrezGeneMapping_proteinCoding_noNa_unique = entrezGeneMapping_proteinCoding_noNa[!duplicated(entrezGeneMapping_proteinCoding_noNa$ensembl_gene_id),]

print(paste('Mapped to entrez and kept only protein coding genes. Remaining #genes:', dim(entrezGeneMapping_proteinCoding_noNa_unique)[1], sep =' '))


ERROR: Error in na.omit(entrezGeneMapping_proteinCoding): object 'entrezGeneMapping_proteinCoding' not found


In [12]:
length(entrezGeneMapping_proteinCoding_noNa_unique$ensembl_gene_id)

[1] 15339

In [9]:
res = gene_ids %in% entrezGeneMapping_proteinCoding_noNa_unique$ensembl_gene_id

In [38]:
data = h5f$matrix
f_name = 'coding_genes_output_hdf5'
gene_names = h5f$gene_attrs$gene_names[res]
gene_ids = h5f$gene_attrs$gene_ids[res]
cell_names = h5f$cell_attrs$cell_names
filtered_matrix = data[res,]

In [40]:
h5createFile(f_name)
h5createGroup(f_name, "cell_attrs")
h5createGroup(f_name, "gene_attrs")
h5write(gene_names, f_name, "gene_attrs/gene_names")
h5write(gene_ids, f_name, "gene_attrs/gene_ids")
h5write(filtered_matrix, f_name,"matrix")
h5write(cell_names, f_name, "cell_attrs/cell_names")

[1] TRUE

[1] TRUE

[1] TRUE

Warning message in h5createDataset(h5loc, name, dim(obj), storage.mode = storage.mode(obj), :
“You created a large dataset with compression and chunking. The chunk size is equal to the dataset dimensions. If you want to read subsets of the dataset, you should test smaller chunk sizes to improve read times. Turn off this warning with showWarnings=FALSE.”

In [41]:
H5Fclose(h5f)